# 世界杯赔率的计算方法

这个 notebook 想要验证的是，赌场在计算赔率的时候，到底是不是用泊松分布作为假设的。

验证过程：通过把比分的赔率里面的 0:0 1:0 2:0 3:0 4:0 5:0 相加，求出Poission

选择 https://www.sporttery.cn/jc/jsq/zqspf/ 中国境内官方体彩的赔率

In [1]:
import requests
import numpy as np
from scipy.stats import poisson, skellam

In [2]:
a = requests.get('https://webapi.sporttery.cn/gateway/jc/football/getMatchCalculatorV1.qry?poolCode=crs&channel=c')
b = a.json()

In [3]:
m = b['value']['matchInfoList'][0]['subMatchList'][0]
d = m['crs']
m['homeRank'],m['homeTeamAbbName'],m['awayTeamAbbName']

('[F组4]', '加拿大', '摩洛哥')

In [4]:
a = np.arange(36).reshape(6,6)

for i in range(6):
    for j in range(6):
        a[i][j] = float(d.get('s{:0>2d}s{:0>2d}'.format(i, j), 1e10))

In [5]:
# 赔率转化为概率
r = 1 / a

In [6]:
p1 = 0
p2 = 0
for i in range(6):
    for j in range(6):
        if i == 0 and r[i][j] > 1e-10:
            # 有百分之1.5的抽水
            p1 += r[i][j] - 0.015
        if j == 0 and r[i][j] > 1e-10:
            p2 += r[i][j] - 0.015

这里统计了 $Poisson(0)$ 的概率

按照 $Poisson(X=k) =  \frac {\lambda ^ {k}}{k} $ $ e^ {-\lambda } $

$Poisson(0) =  e^ {-\lambda } $

可得：$\lambda =  -\ln{Poisson(0)} $


In [7]:
mu1 = -np.log(p1)
mu2 = -np.log(p2)
mu1, mu2

(0.8483168979481747, 1.5501213352776808)

得到 最终比分概率分布 和 半场比分概率分布

In [8]:
matrix = np.zeros((10,10))
for i in range(10):
    for j in range(10):
        matrix[i][j] = poisson(mu1).pmf(i) * poisson(mu2).pmf(j)

半场比分只要计算 $Poisson(\frac {\mu}{2}) $ 即可

In [9]:
matrix_half = np.zeros((10,10))
for i in range(10):
    for j in range(10):
        matrix_half[i][j] = poisson(mu1/2).pmf(i) * poisson(mu2/2).pmf(j)

## 比分玩法

In [10]:
print('比分  估计赔率 - 官方赔率')
for i in range(5):
    for j in range(5):
        ratio = 1 / (matrix[i][j] + 0.02)
        print(i, ':', j, 
              '{:.2f}'.format(ratio),
              '-', 
              '{:.2f}'.format(a[i][j] if a[i][j]<1e5 else np.nan))

比分  估计赔率 - 官方赔率
0 : 0 9.02 - 9.00
0 : 1 6.22 - 6.00
0 : 2 7.74 - 7.00
0 : 3 13.09 - 16.00
0 : 4 23.89 - 40.00
1 : 0 10.30 - 9.00
1 : 1 7.17 - 6.00
1 : 2 8.88 - 7.00
1 : 3 14.74 - 15.00
1 : 4 25.95 - 45.00
2 : 0 18.98 - 20.00
2 : 1 14.15 - 12.00
2 : 2 16.87 - 15.00
2 : 3 24.82 - 26.00
2 : 4 35.89 - 60.00
3 : 0 34.19 - 50.00
3 : 1 29.13 - 32.00
3 : 2 32.15 - 35.00
3 : 3 38.85 - 60.00
3 : 4 45.00 - nan
4 : 0 45.54 - 150.00
4 : 1 43.40 - 80.00
4 : 2 44.73 - 100.00
4 : 3 47.13 - nan
4 : 4 48.85 - nan


**赔率较小时比较一致，赔率越大越不准，是因为赔率大时抽水变少了**

## 计算 胜平负的概率

In [11]:
k = 0
for i in range(10):
    for j in range(10):
        if i > j:
            k += matrix[i][j]
1 / (k + 0.04)

4.073500559985871

In [12]:
k = 0
for i in range(10):
    for j in range(10):
        if i == j:
            k += matrix[i][j]
1 / (k + 0.04)

3.3800326346340763

In [13]:
k = 0
for i in range(10):
    for j in range(10):
        if i < j:
            k += matrix[i][j]
1 / (k + 0.04)

1.7281594810443073

4.07 3.38 1.73 和官网赔率（3.90 3.25 1.74）基本一致

让球玩法同理

## 计算总进球数

In [14]:
k = np.zeros(15)
for i in range(7):
    for j in range(7):
        k[i+j] += matrix[i][j]
1 / (k + 0.02)

array([ 9.02040688,  4.20306725,  3.5544736 ,  4.36810092,  6.88335998,
       12.48528995, 22.71588387, 35.92293235, 45.3429064 , 48.93902165,
       49.81592016, 49.97521836, 49.99773447, 50.        , 50.        ])

9.50 4.25 3.20 3.65 6.00 12.50 21.00 30.00 **（官方赔率）**

9.02 4.20 3.55 4.37 6.88 12.48 22.71, 35.92

基本一致

## 半全场胜平负

先计算半场的概率分布，然后将两个半场叠加起来

In [15]:
from collections import defaultdict
k = defaultdict(lambda: 0)
for i in range(10):
    for j in range(10):
        k[i-j] += matrix_half[i][j]

In [16]:
k

defaultdict(<function __main__.<lambda>()>,
            {0: 0.4089719854781275,
             -1: 0.274190986013359,
             -2: 0.10087530780157698,
             -3: 0.025377386924497928,
             -4: 0.0048385754059646876,
             -5: 0.0007419699619504091,
             -6: 9.510518411010854e-05,
             -7: 1.046906360502788e-05,
             -8: 1.0090941611531775e-06,
             -9: 8.383859760872303e-08,
             1: 0.15005331608995442,
             2: 0.030211321412599614,
             3: 0.004159337617730819,
             4: 0.00043399748261377846,
             5: 3.642071307788139e-05,
             6: 2.554810755422789e-06,
             7: 1.5390573658406238e-07,
             8: 8.118411987629619e-09,
             9: 3.6912700835413905e-10})

In [17]:
def word(num):
    if num > 0:
        return '胜'
    if num == 0:
        return '平'
    if num < 0:
        return '负'

In [18]:
kk = defaultdict(float)
for i in range(-10,10):
    for j in range(-10,10):
        kk[word(i), word(i+j)] += k[i] * k[j]

In [19]:
# 半全场 概率分布
kk

defaultdict(float,
            {('负', '负'): 0.3517990107431333,
             ('负', '平'): 0.04429852335715791,
             ('负', '胜'): 0.010033354836228211,
             ('平', '负'): 0.16609615779192646,
             ('平', '平'): 0.16725808490592176,
             ('平', '胜'): 0.07561773839853625,
             ('胜', '负'): 0.020760550468441864,
             ('胜', '平'): 0.044298523357157914,
             ('胜', '胜'): 0.11983803471341238})

In [20]:
for key in kk.keys():
    print(key, 1 / (kk[key] + 0.02))

('负', '负') 2.689625230581571
('负', '平') 15.552456693994621
('负', '胜') 33.29631356380254
('平', '负') 5.373566073933117
('平', '平') 5.34022336339923
('平', '胜') 10.458310526358447
('胜', '负') 24.533525394222348
('胜', '平') 15.552456693994621
('胜', '胜') 7.151130249001463


半全场赔率：

6.40 15.00 25.00 8.50 4.70 4.25 32.00 15.00 3.05 **（官方赔率）**

6.51 15.14 24.55 9.78 5.30 5.56 32.03 15.14 2.84

差距很小

## 算出所有的 μ

In [21]:
for ii in range(len(b['value']['matchInfoList'])):
    for jj in range(len(b['value']['matchInfoList'][ii]['subMatchList'])):
        m = b['value']['matchInfoList'][ii]['subMatchList'][jj]
        d = m['crs']
        a = np.arange(36).reshape(6,6)

        for i in range(6):
            for j in range(6):
                a[i][j] = float(d.get('s{:0>2d}s{:0>2d}'.format(i, j), 1e10))
        r = 1 / a

        p1 = 0
        p2 = 0
        for i in range(6):
            for j in range(6):
                if i == 0 and r[i][j] > 1e-10:
                    # 有百分之1.5的抽水
                    p1 += r[i][j] - 0.015
                if j == 0 and r[i][j] > 1e-10:
                    p2 += r[i][j] - 0.015

        mu1 = -np.log(p1)
        mu2 = -np.log(p2)
        print(m['homeRank'], m['homeTeamAbbName'], m['awayTeamAbbName'], 
              '{:.2f}'.format(mu1), '{:.2f}'.format(mu2))

[F组4] 加拿大 摩洛哥 0.85 1.55
[F组1] 克罗地亚 比利时 1.16 1.30
[E组2] 日本 西班牙 0.68 2.24
[E组3] 哥斯达 德国 0.47 nan
[H组2] 加纳 乌拉圭 0.72 1.83
[H组3] 韩国 葡萄牙 0.60 2.13
[G组4] 塞尔维亚 瑞士 1.30 1.30
[G组3] 喀麦隆 巴西 0.51 2.91
 荷兰 美国 1.65 0.62
 英格兰 塞内加尔 2.03 0.49


/var/folders/2k/lzky92fs0jq6179hkn52lw7m0000gn/T/ipykernel_17454/155999460.py:23: RuntimeWarning: invalid value encountered in log
  mu2 = -np.log(p2)
